# Manipulación, Exploración y Visualización de Datos

## Preparación de ambiente

### Carga de módulos

In [1]:
# Data Wrangling
import json as js
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Data visualization
import cufflinks as cf
from sklearn.decomposition import PCA

# Preprocessing
from varclushi import VarClusHi
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

cf.go_offline()
pd.set_option("display.max_columns", 50)

import pymysql
pymysql.install_as_MySQLdb()

### Funciones relevantes

In [2]:
def freq_discrete(df, features):
    for feature in features:
        print(f"Feature: {feature}")
        abs_ = df[feature].value_counts(dropna=False).to_frame().rename(columns={feature: "Absolute frequency"})
        rel_ = df[feature].value_counts(dropna=False, normalize= True).to_frame().rename(columns={feature: "Relative frequency"})
        freq = abs_.join(rel_)
        freq["Accumulated frequency"] = freq["Absolute frequency"].cumsum()
        freq["Accumulated %"] = freq["Relative frequency"].cumsum()
        freq["Absolute frequency"] = freq["Absolute frequency"].map(lambda x: "{:,.0f}".format(x))
        freq["Relative frequency"] = freq["Relative frequency"].map(lambda x: "{:,.2%}".format(x))
        freq["Accumulated frequency"] = freq["Accumulated frequency"].map(lambda x: "{:,.0f}".format(x))
        freq["Accumulated %"] = freq["Accumulated %"].map(lambda x: "{:,.2%}".format(x))
        display(freq)

In [3]:
def normalize(df, features):
    for feature in features:
        aux = df[feature].value_counts(True)
        ls_categories = [category for category, freq in aux.items() if freq > 0.05]
        df[feature] = df[feature].map(lambda x: x if x in ls_categories else "Others")
    return df

In [4]:
def get_bounds(x):
    q3 = x.quantile(0.75)
    q1 = x.quantile(0.25)
    iqr = q3 - q1
    lb = q1 - 1.5*iqr
    ub = q3 + 1.5*iqr
    return pd.Interval(lb, ub, closed="both")

## Lectura de Datos

### Lectura de credenciales

In [5]:
with open("cred.json", "r+") as file:
    cred = js.load(file)

In [6]:
db =  cred['db']
host =  cred['host']
port =  cred['port']
flavour = cred['flavour']
username = cred['username']
password = cred['password']

In [7]:
db

'sakila'

### Conexión con base de datos

In [8]:
conn = create_engine(f"{flavour}://{username}:{password}@{host}:{port}/{db}").connect()

### Lectura de datos

In [9]:
df = pd.read_sql(con=conn, sql="""Select * 
                                    From film""")

In [10]:
df.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [11]:
inventory = pd.read_sql(con = conn, sql = "inventory")
payment = pd.read_sql(con = conn, sql = "payment")
rental = pd.read_sql(con = conn, sql = "rental")


film_category = pd.read_sql(con = conn, sql = "film_category")
category = pd.read_sql(con = conn, sql = "category")
language = pd.read_sql(con = conn, sql = "language")
film_actor = pd.read_sql(con = conn, sql = "film_actor")

In [12]:
conn.close()

In [13]:
#merge with inventory
df = pd.merge(df, inventory, on='film_id',how='left')

In [14]:
#merge with rental
df = pd.merge(df, rental, on='inventory_id',how='left')

In [15]:
#merge with payment
df = pd.merge(df, payment, on='rental_id',how='left')

In [16]:
#merge with film_category
df = pd.merge(df, film_category, on='film_id',how='left')
#merge with film_category
df = pd.merge(df, category, on='category_id',how='left')

In [17]:
#merge with language
df = pd.merge(df, language, on='language_id',how='left')

In [18]:
#merge with film_actor
df = pd.merge(df, film_actor, on='film_id',how='left')

In [19]:
df.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_x,inventory_id,store_id,last_update_y,rental_id,rental_date,customer_id_x,return_date,staff_id_x,last_update_x,payment_id,customer_id_y,staff_id_y,amount,payment_date,last_update_y,category_id,last_update_x,name_x,last_update_y,name_y,last_update_x,actor_id,last_update_y
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1.0,1.0,2006-02-15 05:09:17,4863.0,2005-07-08 19:03:15,431.0,2005-07-11 21:29:15,2.0,2006-02-15 21:30:53,11630,431,1,0.99,2005-07-08 19:03:15,2006-02-15 22:18:35,6,2006-02-15 05:07:09,Documentary,2006-02-15 04:46:27,English,2006-02-15 05:02:19,1.0,2006-02-15 05:05:03
1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1.0,1.0,2006-02-15 05:09:17,4863.0,2005-07-08 19:03:15,431.0,2005-07-11 21:29:15,2.0,2006-02-15 21:30:53,11630,431,1,0.99,2005-07-08 19:03:15,2006-02-15 22:18:35,6,2006-02-15 05:07:09,Documentary,2006-02-15 04:46:27,English,2006-02-15 05:02:19,10.0,2006-02-15 05:05:03
2,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1.0,1.0,2006-02-15 05:09:17,4863.0,2005-07-08 19:03:15,431.0,2005-07-11 21:29:15,2.0,2006-02-15 21:30:53,11630,431,1,0.99,2005-07-08 19:03:15,2006-02-15 22:18:35,6,2006-02-15 05:07:09,Documentary,2006-02-15 04:46:27,English,2006-02-15 05:02:19,20.0,2006-02-15 05:05:03
3,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1.0,1.0,2006-02-15 05:09:17,4863.0,2005-07-08 19:03:15,431.0,2005-07-11 21:29:15,2.0,2006-02-15 21:30:53,11630,431,1,0.99,2005-07-08 19:03:15,2006-02-15 22:18:35,6,2006-02-15 05:07:09,Documentary,2006-02-15 04:46:27,English,2006-02-15 05:02:19,30.0,2006-02-15 05:05:03
4,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1.0,1.0,2006-02-15 05:09:17,4863.0,2005-07-08 19:03:15,431.0,2005-07-11 21:29:15,2.0,2006-02-15 21:30:53,11630,431,1,0.99,2005-07-08 19:03:15,2006-02-15 22:18:35,6,2006-02-15 05:07:09,Documentary,2006-02-15 04:46:27,English,2006-02-15 05:02:19,40.0,2006-02-15 05:05:03


In [20]:
df.columns

Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update_x',
       'inventory_id', 'store_id', 'last_update_y', 'rental_id', 'rental_date',
       'customer_id_x', 'return_date', 'staff_id_x', 'last_update_x',
       'payment_id', 'customer_id_y', 'staff_id_y', 'amount', 'payment_date',
       'last_update_y', 'category_id', 'last_update_x', 'name_x',
       'last_update_y', 'name_y', 'last_update_x', 'actor_id',
       'last_update_y'],
      dtype='object')

#### observando periodo de los datos

In [21]:
#Check dates
df["rental_date"].describe()

<ipython-input-21-960fc6004a56>:2: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



count                   88020
unique                  15815
top       2006-02-14 15:16:03
freq                      999
first     2005-05-24 22:53:30
last      2006-02-14 15:16:03
Name: rental_date, dtype: object

**Se decide agrupar semanalmente**

### Agrupación

In [22]:
df_complete = df.groupby(['title','film_id', pd.Grouper(key='rental_date', freq='W-MON'),#group by these
           'description', 'release_year','length','rating', 'special_features', #preserve these
            'name_y','name_x',
            'replacement_cost', 'rental_duration', 'rental_rate'],
          )\
        .aggregate(
            {
                'rental_id':'nunique', 'amount':'mean', 'store_id':'nunique', 'actor_id':'nunique'}
        )\
        .reset_index()\
        .rename(
            columns = {'name_y':'language', 'name_x':'category',
                       'rental_id':'rental_count', 'amount':'payment$_mean', 'store_id':'store_count',
                       'actor_id':'actor_count'
                      }
        )\
        

In [23]:
df_complete.head()

,title,film_id,rental_date,description,release_year,length,rating,special_features,language,category,replacement_cost,rental_duration,rental_rate,rental_count,payment$_mean,store_count,actor_count
0,ACADEMY DINOSAUR,1,2005-05-30,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,2,1.49,2,10
1,ACADEMY DINOSAUR,1,2005-06-20,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,2,0.99,2,10
2,ACADEMY DINOSAUR,1,2005-06-27,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,1,1.99,1,10
3,ACADEMY DINOSAUR,1,2005-07-11,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,4,0.99,2,10
4,ACADEMY DINOSAUR,1,2005-08-01,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,5,1.39,2,10


**se crean registros para las semanas faltantes (semanas en las que no se rentaron esas películas)**

In [24]:
rental_date = pd.date_range(min(df_complete.rental_date), max(df_complete.rental_date), freq='W-MON')
title = df_complete['title'].unique()
idx = pd.MultiIndex.from_product((title, rental_date), names=['title', 'rental_date'])

In [25]:
df_complete = df_complete.set_index(['title', 'rental_date']).reindex(idx, fill_value=0).reset_index()
df_complete

,title,rental_date,film_id,description,release_year,length,rating,special_features,language,category,replacement_cost,rental_duration,rental_rate,rental_count,payment$_mean,store_count,actor_count
0,ACADEMY DINOSAUR,2005-05-30,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,2,1.49,2,10
1,ACADEMY DINOSAUR,2005-06-06,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0
2,ACADEMY DINOSAUR,2005-06-13,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0
3,ACADEMY DINOSAUR,2005-06-20,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,2,0.99,2,10
4,ACADEMY DINOSAUR,2005-06-27,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,1,1.99,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37357,ZORRO ARK,2006-01-23,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0
37358,ZORRO ARK,2006-01-30,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0
37359,ZORRO ARK,2006-02-06,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0
37360,ZORRO ARK,2006-02-13,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0


**Se rellenan las filas vacías**  
dejando los valores de rental_count en 0

In [27]:
for index, row in df_complete.iterrows():
    if row['film_id'] == 0:
        aux1 = df_complete[df_complete['title'] == row['title']]
        aux2 = aux1[aux1['film_id'] != 0].head(1).reset_index(drop=True)
        df_complete.loc[index,:] = aux2.loc[0,:].values
        df_complete.loc[[index],['rental_date']] = row['rental_date']
        df_complete.loc[[index],['rental_count']] = 0

In [31]:
df_complete

,title,rental_date,film_id,description,release_year,length,rating,special_features,language,category,replacement_cost,rental_duration,rental_rate,rental_count,payment$_mean,store_count,actor_count
0,ACADEMY DINOSAUR,2005-05-30,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,2,1.49,2,10
1,ACADEMY DINOSAUR,2005-06-06,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,0,1.49,2,10
2,ACADEMY DINOSAUR,2005-06-13,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,0,1.49,2,10
3,ACADEMY DINOSAUR,2005-06-20,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,2,0.99,2,10
4,ACADEMY DINOSAUR,2005-06-27,1,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,"Deleted Scenes,Behind the Scenes",English,Documentary,20.99,6,0.99,1,1.99,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37357,ZORRO ARK,2006-01-23,1000,A Intrepid Panorama of a Mad Scientist And a B...,2006,50,NC-17,"Trailers,Commentaries,Behind the Scenes",English,Comedy,18.99,3,4.99,0,4.99,1,3
37358,ZORRO ARK,2006-01-30,1000,A Intrepid Panorama of a Mad Scientist And a B...,2006,50,NC-17,"Trailers,Commentaries,Behind the Scenes",English,Comedy,18.99,3,4.99,0,4.99,1,3
37359,ZORRO ARK,2006-02-06,1000,A Intrepid Panorama of a Mad Scientist And a B...,2006,50,NC-17,"Trailers,Commentaries,Behind the Scenes",English,Comedy,18.99,3,4.99,0,4.99,1,3
37360,ZORRO ARK,2006-02-13,1000,A Intrepid Panorama of a Mad Scientist And a B...,2006,50,NC-17,"Trailers,Commentaries,Behind the Scenes",English,Comedy,18.99,3,4.99,0,4.99,1,3


In [29]:
df_complete.to_csv('data.csv', index=False)